<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Faceplusplus/ElaboraFoto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install clarifai

In [0]:
#Da cambiare:
#nome del csv e cartella da cui prendo le foto (foto ordinate come "ultima modifica e freccetta in giù")

import json
import csv
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive

# apro e parso il file .json
def convert(response):

  with open('utenti.txt', 'w') as outfile:
    json.dump(response, outfile) 
        
  # apro e parso il file .json
  data = open('utenti.txt').read()
  data_parsed = json.loads(data)

  #creo e apro un file .csv per scrivere
  file_parsed = open('HarumiSushi_10%Random_UsersData.csv', 'w')
    
  #creo il writer
  writer = csv.writer(file_parsed)

  #creo header del csv
  header = ['id', 'age', 'gender', 'ethnicity']
  writer.writerow(header)

  #ciclo per estrarre dal file json i valori di id, età, genere ed etnia
  for person in data_parsed['user']:
     writer.writerow([person['id'],
                      person['age'],
                      person['gender'],
                      person['ethnicity']])
     
  
  #salvo e chiudo il file .csv
  file_parsed.close()

  folder_id = '1wfZ0d3cTufosLRhPKkShf7kQbJlUbBvH'    #cartella dove metto il csv
  file_metadata = {'title': 'HarumiSushi_10%Rabdom_UsersData.csv',"parents":[{"id": folder_id,"kind":"drive#childList"}]}
  folder = drive.CreateFile(file_metadata)
  folder.SetContentFile('HarumiSushi_10%Random_UsersData.csv') #The contents of the file
  folder.Upload()

In [0]:
from clarifai.rest import ClarifaiApp
import time

def detect_clarifai(id, url_photo, data):
  app = ClarifaiApp(api_key='6651c2b2e8d64d3291238ef15277c18c')
  model = app.models.get(model_id="c0c0ac362b03416da06ab3fa36fb58e3")
  response = model.predict_by_url(url = '%s' % url_photo)
  
  if response['outputs'][0]['data'] != {}:
    x = 0
    while (x < len(response['outputs'][0]['data']['regions'])):
      age = response['outputs'][0]['data']['regions'][x]['data']['concepts'][0]['name']
      gender = response['outputs'][0]['data']['regions'][x]['data']['concepts'][20]['name']
      ethnicity = response['outputs'][0]['data']['regions'][x]['data']['concepts'][22]['name']

      data['user'].append({
          'id' : '%s' % id,
          'age' : '%s' % age,
          'gender' : '%s' % gender,
          'ethnicity' : '%s' % ethnicity,
      })
      x = x + 1
  else:
    return

In [22]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
from clarifai.rest import ApiError
import json

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Auto-iterate through all files that matches this query
file_list = drive.ListFile({'q': "'1EdhoU-1Kqj8Ztf19NanbEYGK2UA402lo' in parents and trashed=false"}).GetList()    #cartella da cui prendo le foto

data = {}
data['user'] = []
for file1 in file_list:
  id = file1['title'] 
  detect_clarifai(id[0:22], file1['thumbnailLink'], data)

convert(data)

ApiError: ignored